# Part 1

#### Completing `bust_probability` function

In [3]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
import torch.optim as optim
import sklearn
from sklearn import metrics as metrics
import pandas as pd

In [4]:
def data_loader(path, table_idx, player_or_dealer):
    #utility for loading train.csv, example use in the notebook
    data = pd.read_csv(path, header=[0,1,2])
    spy = data[(f'table_{table_idx}', player_or_dealer, 'spy')]
    card = data[(f'table_{table_idx}', player_or_dealer, 'card')]
    return np.array([spy, card]).T

In [5]:
def bust_probability(dealer_cards):
    """
    dealer_cards: list -> integer series of player denoting value of cards observed upto this point

    Current body is random for now, change it accordingly
    
    output: probability of going bust on this table
    """
    bust_count = 0
    total_cards = len(dealer_cards)
    score = 0
    for card in dealer_cards:
        score += card
        if score > 16:
            if score > 21:
                bust_count += 1
            score = 0
    return bust_count/total_cards

In [6]:
for table_idx in range(0, 5):
    dealer_data = data_loader("data/train.csv", table_idx, "dealer")
    dealer_cards = dealer_data[:,1]
    print(table_idx, bust_probability(dealer_cards))

0 0.18865
1 0.10155
2 0.09175
3 0.07995
4 0.00055


### Two additional points for "willingness to play"

### **1. Effective House Edge (EHE)**  
#### **Definition:**  
This metric estimates the actual house edge at a given table based on observed outcomes rather than theoretical expectations. It measures the average percentage of a player’s bet lost per round.  

#### **Formula:**  
$$
EHE = 1 - \frac{\text{Total player winnings}}{\text{Total player bets}}
$$
where:  
- **Total player winnings** = sum of all amounts won by players.  
- **Total player bets** = sum of all bets placed by players.  

#### **Strengths:**  
- Captures the real-world profitability of a table.  
- Accounts for both dealer performance and player behavior.  

#### **Weaknesses:**  
- Requires tracking actual bet sizes and payouts.  
- Can be skewed if a few players make bad decisions.  

---

### **2. Volatility Score (VS)**  
#### **Definition:**  
Measures how unpredictable the dealer’s bust rate is across different hands.  
A high volatility score suggests that the dealer’s performance is inconsistent.

#### **Formula:**  
$$
VS = \sigma_b = \sqrt{\frac{1}{N} \sum_{i=1}^{N} (B_i - \mu_b)^2}
$$
where:  
- \( $B_i$ \) is the bust probability observed in past hands.  
- \( $\mu_b$ \) is the mean bust probability.  
- \( $\sigma_b$ \) is the standard deviation of bust probabilities.

#### **Strengths:**  
- Identifies tables with consistent dealer behavior (lower volatility is preferable).  
- Helps risk-averse players avoid unpredictable situations.

#### **Weaknesses:**  
- Doesn’t directly indicate profitability.  
- Requires enough historical data for accurate estimation.

# Part 2